Question 2 part a

In [0]:
import re
from operator import add
from pyspark import SparkContext
import numpy as np

# File location and type
file_location = "/FileStore/tables/city_temperature.csv"
file_type = "csv"

# loading input file into the RDD
city_rdd = sc.textFile("/FileStore/tables/city_temperature.csv")
# city_rdd.collect()

#PREPROCESSING

# filter out the header
head = city_rdd.first()
rdd1 = city_rdd.filter(lambda x: x != head)
#rdd1.collect()

map_rdd = rdd1.map(lambda x:x.split(",")) \
              .map(lambda c: (c[0], float(c[7]))) #mapping key value pairs where Region = key and AvgTemp = value
#map_rdd.collect()

# getting temperatures of each region
temps_rdd = map_rdd.reduceByKey(lambda x, y: x+y)
#temps_rdd.collect()

# get counts of regions
counts = map_rdd.countByKey() #int dict

temps_rdd1 = temps_rdd.sortByKey()

#converting counts into an array
counts_data = list(counts.items())
counts_arr = np.array(counts_data)

#convert counts_arr into an RDD
counts_rdd = sc.parallelize(counts_arr)

#joining the 2 rdds
join_rdd = temps_rdd1.join(counts_rdd).sortByKey()
#join_rdd.collect()

#getting the avg temp of each region and displaying them
display(join_rdd.mapValues(lambda x: float(x[0])/float(x[1])).collect())


_1,_2
Africa,53.54951656193528
Asia,62.56865184754641
Australia/South Pacific,61.180869127275976
Europe,46.69628524306878
Middle East,68.38455378399779
North America,55.300932625245935
South/Central America & Carribean,62.189438801074665


Question 2 part b

In [0]:
import re
from operator import add
from pyspark import SparkContext
import numpy as np

# File location and type
file_location = "/FileStore/tables/city_temperature.csv"
file_type = "csv"

# loading input file into the RDD
city_rdd = sc.textFile("/FileStore/tables/city_temperature.csv")
# city_rdd.collect()

#PREPROCESSING
# filter out the header
head = city_rdd.first()
rdd1 = city_rdd.filter(lambda x: x != head)
#rdd1.collect()

#get rid of all regions that are not Asia
asia_rdd = rdd1.filter(lambda x:x.split(",")[0] == "Asia")
#asia_rdd.collect()

# making key value pairs
kv_rdd = asia_rdd.map(lambda x: x.split(",")) \
                 .map(lambda c: ((c[6], c[1]), float(c[7]))) # key = (year, countries) and values = avg temp
#kv_rdd.collect()

# add all the temps from the same year 
reduced_rdd = kv_rdd.reduceByKey(lambda a, b: (a+b))
#reduced_rdd.collect()

# get count of each year
counts_by_key = kv_rdd.countByKey() # int dict that needs to be converted to rdd

# converting counts into an array
counts_data = list(counts_by_key.items())
counts_arr = np.array(counts_data)

# convert counts_arr into an RDD
counts_rdd = sc.parallelize(counts_arr)
#counts_rdd.collect()

# joining the 2 rdds with aggregated temps and counts
join_rdd = reduced_rdd.join(counts_rdd).sortByKey()
#join_rdd.collect()

# divide sum of temps by count of each year
display(join_rdd.mapValues(lambda x: float(x[0])/float(x[1])).collect())


<command-4322430461351686>:38: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  counts_arr = np.array(counts_data)


_1,_2
"List(1995, Bangladesh)",42.21424657534248
"List(1995, China)",59.772493150684916
"List(1995, Hong Kong)",74.12
"List(1995, India)",79.12102739726028
"List(1995, Indonesia)",49.03205479452054
"List(1995, Japan)",55.523652968036615
"List(1995, Kazakhstan)",48.61095890410961
"List(1995, Laos)",81.52958904109589
"List(1995, Malaysia)",80.56465753424658
"List(1995, Mongolia)",31.9356164383562


Question 2 part c

In [0]:
import re
from operator import add
from pyspark import SparkContext
import numpy as np

# File location and type
file_location = "/FileStore/tables/city_temperature.csv"
file_type = "csv"

# loading input file into the RDD
city_rdd = sc.textFile("/FileStore/tables/city_temperature.csv")
# city_rdd.collect()

#PREPROCESSING
# filter out the header
head = city_rdd.first()
rdd1 = city_rdd.filter(lambda x: x != head)
#rdd1.collect()

#get rid of all regions that are not Asia
spain_rdd = rdd1.filter(lambda x:x.split(",")[1] == "Spain")
#spain_rdd.collect()

# making key value pairs
kv_rdd = spain_rdd.map(lambda x: x.split(",")) \
                 .map(lambda c: ((c[3]), float(c[7]))) # key = city and values = avg temp
#kv_rdd.collect()

# add all the temps from the same year 
reduced_rdd = kv_rdd.reduceByKey(lambda a, b: (a+b))
# reduced_rdd.collect()

# get count of each year
counts_by_key = kv_rdd.countByKey() # int dict that needs to be converted to rdd

# converting counts into an array
counts_data = list(counts_by_key.items())
counts_arr = np.array(counts_data)

# convert counts_arr into an RDD
counts_rdd = sc.parallelize(counts_arr)
#counts_rdd.collect()

# joining the 2 rdds with aggregated temps and counts
join_rdd = reduced_rdd.join(counts_rdd).sortByKey()
# join_rdd.collect()

# divide sum of temps by count of each year
display(join_rdd.mapValues(lambda x: float(x[0])/float(x[1])).collect())

_1,_2
Barcelona,61.26926397582565
Bilbao,58.69515432764961
Madrid,58.4405352903085


Question 2 part d

approach 3

In [0]:
import re
from operator import add
from pyspark import SparkContext
import numpy as np
from pyspark.sql.functions import *

#reading in the files
city_temp = spark.read.option("header", "true").csv("/FileStore/tables/city_temperature.csv")
country_list = spark.read.option("header", "true").csv("/FileStore/tables/country_list-2.csv")

# city_temp.collect()
# country_list.collect()

joined = city_temp.join(country_list, "Country", "inner")
#joined.collect()

#averaged all the avgtemps for each (country, capital) pair, then aggregating them
result = joined.groupBy("Country", "Capital").agg(avg("AvgTemperature")) 
display(result)

Country,Capital,avg(AvgTemperature)
Benin,Porto-Novo,76.15212605223395
India,New Delhi,79.76246121468796
Namibia,Windhoek,57.990341031729066
China,Beijing,59.98367545166184
Nepal,Kathmandu,49.4923475445224
Kazakhstan,Astana,49.31983810037769
Tanzania,Dodoma,65.30484079173841
Mauritania,Nouakchott,73.41478705686403
Sierra Leone,Freetown,-9.820260911579975
Tunisia,Tunis,66.47864234837016
